In [2]:
# 1. 從 github 下載資料
!git clone https://github.com/cyiping/RAG-20241226.git

Cloning into 'RAG-20241226'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 16.95 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
# 2. 切換工作目錄
import os
os.chdir('/kaggle/working/RAG-20241226')
!ls

rag-2024-12-26.ipynb  柯文哲起訴書-2024-12-26.pdf
README.md	      高虹安-李正皓-113年度訴字第27號.pdf


In [4]:
# 3. 安裝套件
!pip install PyPDF2
!pip install sentence-transformers faiss-cpu transformers
print("---- ok !")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 47.9 MB/s eta 0:00:00:00:0100:01
---- ok !


### RAG 程式的設計理念（適用於 Kaggle）
#### 1. 資料預處理：
- 使用 PyPDF2 或 pdfplumber 從上傳的 PDF 檔案中提取文字。
- 清理並將提取的文字進行分詞，以便後續處理。

#### 2. 索引建立：
- 使用 FAISS 等向量檢索工具，透過 SentenceTransformers 模型生成文本嵌入。
- 將這些嵌入儲存以進行高效的相似度搜尋。

#### 3. 文本檢索：
- 使用 FAISS 根據使用者的查詢檢索相關的文本區塊。

#### 4. 答案生成：
- 使用預訓練的語言模型（例如 OpenAI GPT 或 Hugging Face 模型）根據檢索到的區塊生成答案。

#### 5. 流程整合：
- 將檢索與生成步驟整合，完成一個流暢的 RAG 管線。

# 採用facebook/mbart-large-50

In [5]:
import os
import re
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 從 PDF 提取文字
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# 將文字分割為區塊
def split_text_into_chunks(text, chunk_size=500):
    sentences = text.split('。')  # 使用適用於繁體中文的句號進行分割
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + '。'
        else:
            chunks.append(chunk)
            chunk = sentence + '。'
    if chunk:
        chunks.append(chunk)
    return chunks

# 對文字區塊進行嵌入
def embed_chunks(chunks, model):
    return model.encode(chunks, convert_to_tensor=True)

# 建立 FAISS 索引
def create_faiss_index(embeddings):
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.cpu().numpy())
    return index

# 增強的清理上下文函數
def clean_context(context):
    # 移除數字、符號及多餘的空白
    cleaned_context = re.sub(r'\d+|[{}\[\],\'";:]', '', context)
    cleaned_context = re.sub(r'\s+', ' ', cleaned_context).strip()
    return cleaned_context

# 檢索相關的區塊
def retrieve_chunks(query, index, model, chunks, top_k=5):
    query_embedding = model.encode([query], convert_to_tensor=True)
    distances, indices = index.search(query_embedding.cpu().numpy(), top_k)
    retrieved_chunks = [chunks[i] for i in indices[0] if i < len(chunks)]
    
    # 過濾空或無意義區塊
    filtered_chunks = [chunk for chunk in retrieved_chunks if len(chunk.strip()) > 10]
    if not filtered_chunks:
        return ["查無相關內容"]
    
    return filtered_chunks

# 生成答案
def generate_answer(query, context, tokenizer, model):
    # 限制上下文長度避免超過模型限制
    max_context_length = 1000  # 避免超過模型的最大 token 限制
    context = context[:max_context_length]
    
    input_text = f"context: {context} question: {query}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(inputs, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# RAG 主流程
def rag_pipeline(pdf_path, query):
    # 步驟 1：提取並預處理
    text = extract_text_from_pdf(pdf_path)
    chunks = split_text_into_chunks(text)

    # 步驟 2：嵌入與索引
    embed_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embed_chunks(chunks, embed_model)
    index = create_faiss_index(embeddings)

    # 步驟 3：檢索相關區塊
    relevant_chunks = retrieve_chunks(query, index, embed_model, chunks)

    # 清理上下文
    context = clean_context(" ".join(relevant_chunks))

    # 步驟 4：生成答案
    tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50')
    gen_model = AutoModelForSeq2SeqLM.from_pretrained('facebook/mbart-large-50')
    tokenizer.src_lang = "zh_CN"  # 設置源語言為中文
    answer = generate_answer(query, context, tokenizer, gen_model)

    return answer

# 使用範例
pdf_path = '高虹安-李正皓-113年度訴字第27號.pdf'  # 替換為您的 PDF 檔案路徑
query = "判決的關鍵理由是什麼？"
answer = rag_pipeline(pdf_path, query)
print(answer)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

context: (續上頁) 第二十八頁、李正皓:第一個發言人你先回一 下記者的訊息,發言人的爭議我 們可能晚點聊,發言人現在連專 門挖弊案的大記者的訊息從去年 未讀到現在,發言人先把記者的 問題回一回再說,還嫌人家問題 太多。 又被證 該資料,其內記載A類即協議合建都更案,審議時間最短 約.年(見本院卷二第頁),亦較系爭昇益案進行所 花費之約年個月時間為長。 、次查,被證於年月日之媒體報導,載稱於年 月日,台北地檢署偵辦高虹安涉詐領助理費案,傳喚高虹 安、男友李忠庭等(見本院


In [6]:
query = "李正皓的判決是？"
answer = rag_pipeline(pdf_path, query)
print(answer)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

context: (續上頁) 第二十八頁、李正皓:第一個發言人你先回一 下記者的訊息,發言人的爭議我 們可能晚點聊,發言人現在連專 門挖弊案的大記者的訊息從去年 未讀到現在,發言人先把記者的 問題回一回再說,還嫌人家問題 太多。 、次查,被證於年月日之媒體報導,載稱於年 月日,台北地檢署偵辦高虹安涉詐領助理費案,傳喚高虹 安、男友李忠庭等(見本院卷一第頁),已報導提及李 忠庭係原告高虹安之男友,而此亦為原告在本件所未爭執。 、李正皓:奇怪?好像高虹安市府 講大家都是笨蛋,解凍就是一種


# 採用 uer/t5-base-chinese-cluecorpussmall

In [9]:
import os
import re
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 從 PDF 提取文字
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            # 移除換行符並合併行
            page_text = re.sub(r'\n', '', page_text)
            text += page_text
    return text

# 將文字分割為區塊
def split_text_into_chunks(text, chunk_size=500):
    sentences = text.split('。')  # 使用適用於繁體中文的句號進行分割
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + '。'
        else:
            chunks.append(chunk)
            chunk = sentence + '。'
    if chunk:
        chunks.append(chunk)
    return chunks

# 對文字區塊進行嵌入
def embed_chunks(chunks, model):
    return model.encode(chunks, convert_to_tensor=True)

# 建立 FAISS 索引
def create_faiss_index(embeddings):
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.cpu().numpy())
    return index

# 增強的清理上下文函數
def clean_context(context):
    # 僅保留中文文字與基本標點符號（如句號、逗號）
    cleaned_context = re.sub(r'[^一-龥。，！？]', '', context)
    cleaned_context = re.sub(r'\s+', ' ', cleaned_context).strip()
    return cleaned_context

# 檢索相關的區塊
def retrieve_chunks(query, index, model, chunks, top_k=5):
    query_embedding = model.encode([query], convert_to_tensor=True)
    distances, indices = index.search(query_embedding.cpu().numpy(), top_k)
    retrieved_chunks = [chunks[i] for i in indices[0] if i < len(chunks)]
    
    # 增加關鍵詞過濾，確保相關性
    filtered_chunks = [chunk for chunk in retrieved_chunks if query in chunk]
    if not filtered_chunks:
        return ["查無相關內容"]
    
    return filtered_chunks

# 生成答案
def generate_answer(query, context, tokenizer, model):
    # 限制上下文長度避免超過模型限制
    max_context_length = 1000
    context = context[:max_context_length]
    
    input_text = f"問題：{query} 上下文：{context}"
    print("生成模型輸入:", input_text)  # Debug 輸入文本
    
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(inputs, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# RAG 主流程
def rag_pipeline(pdf_path, query):
    # 步驟 1：提取並預處理
    text = extract_text_from_pdf(pdf_path)
    chunks = split_text_into_chunks(text)

    # 步驟 2：嵌入與索引
    embed_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embed_chunks(chunks, embed_model)
    index = create_faiss_index(embeddings)

    # 步驟 3：檢索相關區塊
    relevant_chunks = retrieve_chunks(query, index, embed_model, chunks)

    # 清理上下文
    context = clean_context(" ".join(relevant_chunks))
    print("清理後的上下文:", context)  # Debug 清理後的上下文

    # 步驟 4：生成答案
    tokenizer = AutoTokenizer.from_pretrained('uer/t5-base-chinese-cluecorpussmall')
    gen_model = AutoModelForSeq2SeqLM.from_pretrained('uer/t5-base-chinese-cluecorpussmall')
    answer = generate_answer(query, context, tokenizer, gen_model)

    return answer

# 使用範例
pdf_path = '高虹安-李正皓-113年度訴字第27號.pdf'  # 替換為您的 PDF 檔案路徑
query = "判決的關鍵理由是什麼？"
answer = rag_pipeline(pdf_path, query)
print("最終答案:", answer)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

清理後的上下文: 查無相關內容
生成模型輸入: 問題：判決的關鍵理由是什麼？ 上下文：查無相關內容
最終答案: extra0


In [12]:
query = "總結這份文件"
answer = rag_pipeline(pdf_path, query)
print("最終答案:", answer)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

清理後的上下文: 查無相關內容
生成模型輸入: 問題：總結這份文件 上下文：查無相關內容
最終答案: extra0 extra2
